### Studying the parameter space metric density for Sage

Refer: Gravitational waves from inspiralling compact binaries: hexagonal template
placement and its efficiency in detecting physical signals by Thomas Cokelaer. 

Link = https://arxiv.org/abs/0706.4437

Salient idea - Since the model h explicitly depends on the two mass parameters M and η, then the spacing dxi are
function of these two quantities as well. However, the metric expressed in these two coordinates is 
not a constant; it is not a constant either if we were to use the component masses, m1 and m2. 
The preference of chirptimes, denoted τ0 and τ3 (see appendix B, Eqs. B1) as coordinates on the signal 
manifold is indeed more practical because these variables are almost Cartesian. Although not 
perfectly constant for PN-order larger than 1PN, we shall assume that the metric is essentially 
constant in the local vicinity of every point on the manifold. We could use any combinations of 
chirptimes, but using the pair τ0 −τ3, there exists analytical inversion with the pair M − η.

In [132]:
import numpy as np
import lal
import matplotlib.pyplot as plt

In [177]:
# Constants
G = 6.67e-11
c = 3.0e8

In [182]:
# Conversions make sense
print(lal.MTSUN_SI)
print(1.989e30*G/c**3.)

4.925490947641267e-06
4.9135666666666664e-06


#### Tau0 and Tau3

In [206]:
# Define tau0 and tau3
fL = 40.0 # Hz
A0 = 5/(256.*(np.pi*fL)**(8./3.))
tau0 = lambda mchirp: A0 * (mchirp*1.989e30*G/c**3.)**(-5./3.)

A3 = np.pi/(8.*(np.pi*fL)**(5./3.))
tau3 = lambda _tau0, M: (A3/A0)*_tau0*(M*1.989e30*G/c**3.)

In [207]:
# Limits on chirp mass
ml = 3.0 # Msun
mu = 30.0 # Msun
min_mchirp = (ml*ml / (ml+ml)**2.)**(3./5) * (ml + ml)
max_mchirp = (mu*mu / (mu+mu)**2.)**(3./5) * (mu + mu)

# Bounds on total mass
min_total_mass = ml*2.
max_total_mass = mu*2.

In [208]:
# Place limits on tau0
min_tau0 = tau0(max_mchirp)
max_tau0 = tau0(min_mchirp)
# Place limits on tau3
min_tau3 = tau3(min_tau0, min_total_mass)
max_tau3 = tau3(max_tau0, max_total_mass)

print('Limits on tau0 = {}, {}'.format(min_tau0, max_tau0))
print('Limits on tau3 = {}, {}'.format(min_tau3, max_tau3))

Limits on tau0 = 0.15100229716958533, 7.00890576392226
Limits on tau3 = 0.011247897968794487, 5.220811761357236


#### A0 and A3 for calculating Tau0 and Tau3

In [209]:
# VERIFIED
def _a0(f_lower):
    """ Used in calculating chirp times: see Cokelaer, arxiv.org:0706.4437
        appendix 1, also lalinspiral/python/sbank/tau0tau3.py. """
    return 5. / (256. * (np.pi * f_lower)**(8./3.))

# VERIFIED
def _a3(f_lower):
    """ Another parameter used for chirp times """
    return np.pi / (8. * (np.pi * f_lower)**(5./3.))

#### Total mass and eta from tau0 and tau3

In [213]:
# VERIFIED
def mtotal_from_tau0_tau3(tau0, tau3, f_lower, in_seconds=False):
    """ Returns total mass from :math: tau_0, tau_3 """
    mtotal = (tau3 / _a3(f_lower)) / (tau0 / _a0(f_lower)) # in seconds
    if in_seconds:
        return mtotal
    # convert to kgs
    mtotal /= (1.989e30*G/c**3.)
    return mtotal

# VERIFIED
def _eta_from_tau0_tau3(tau0, tau3, f_lower):
    """ Returns symmetric mass ratio from :math: tau_0, tau_3 """
    mtotal = mtotal_from_tau0_tau3(tau0, tau3, f_lower, in_seconds=True) # in seconds
    eta = mtotal**(-2./3.) * (_a3(f_lower) / tau3)
    return eta

# VERIFIED
def eta_from_tau0_tau3(tau0, tau3, f_lower):
    """ Returns symmetric mass ratio from :math: tau_0, tau_3 """
    eta = ((tau3/_a3(f_lower)) * (_a0(f_lower)/tau0)**(2./5.))**(-5./3.) # dimensionless
    return eta

In [214]:
# Check boundary conditions
# These values are swapped on Figure 1 in https://arxiv.org/abs/0706.4437. Probably a typo.
print(mtotal_from_tau0_tau3(min_tau0, min_tau3, 40.0)) # This should be around 14 Msun
print(mtotal_from_tau0_tau3(max_tau0, max_tau3, 40.0)) # This should be around 100 Msun


6.0
59.99999999999999


In [215]:
print(eta_from_tau0_tau3(min_tau0, min_tau3, 40.0))
print(_eta_from_tau0_tau3(min_tau0, min_tau3, 40.0))

11.603972084031948
11.603972084031936


#### mass1 and mass2 from tau0 and tau3

In [192]:
def mass1_from_mtotal_eta(mtotal, eta):
    """ Returns the primary mass from the total mass and symmetric mass ratio. """
    return 0.5 * mtotal * (1.0 + (1.0 - 4.0 * eta)**0.5)

def mass2_from_mtotal_eta(mtotal, eta):
    """ Returns the secondary mass from the total mass and symmetric mass ratio. """
    return 0.5 * mtotal * (1.0 - (1.0 - 4.0 * eta)**0.5)

In [193]:
def mass1_from_tau0_tau3(tau0, tau3, f_lower):
    """ Returns the primary mass from the given :math: tau_0, tau_3 """
    mtotal = mtotal_from_tau0_tau3(tau0, tau3, f_lower) # in Msun
    eta = eta_from_tau0_tau3(tau0, tau3, f_lower) # dimensionless
    return mass1_from_mtotal_eta(mtotal, eta) # in Msun

def mass2_from_tau0_tau3(tau0, tau3, f_lower):
    """ Returns the secondary mass from the given :math: tau_0, tau_3 """
    mtotal = mtotal_from_tau0_tau3(tau0, tau3, f_lower) # in Msun
    eta = eta_from_tau0_tau3(tau0, tau3, f_lower) # dimensionless
    return mass2_from_mtotal_eta(mtotal, eta) # in Msun

#### Sampling uniformly on tau0 and tau3

In [204]:
# This is what happens inside m1 from tau0 and tau3 given f_lower
mtotal = mtotal_from_tau0_tau3(min_tau0, min_tau3, 20.0) # This doesn't makes sense (should be around 100 Msun)
eta = eta_from_tau0_tau3(min_tau0, min_tau3, 20.0) # This doesn't make sense 
m1 = mass1_from_mtotal_eta(mtotal, eta) # should be around 50.0 Msun
print(mtotal, eta, m1) 

14.0 0.25 7.0


In [195]:
# This should correspond to m1=m2=m_min
print(mass1_from_tau0_tau3(min_tau0, min_tau3, 20.0))
# This should correspond to m1=m2=m_max
print(mass1_from_tau0_tau3(min_tau0, max_tau3, 20.0))

(7.000000000000002+35.34294166396984j)
2646.415537853632


In [131]:
eta = lambda m1, m2: m1*m2/(m1+m2)**2
mass1 = np.random.uniform(7.0, 50.0, 100_000)
mass2 = np.random.uniform(7.0, 50.0, 100_000)
etas = np.array([eta(_m1, _m2) for _m1, _m2 in zip(mass1, mass2)])